In [37]:
%pylab qt
import csv
#to change working directory
#import os
#os.chdir(r'D:\!Data\2015_09_22')

filename = 'tek00024.csv'

#only for 2 coloms
x = []
y = []
with open(filename, 'r') as csvfile:
    a = csv.reader(csvfile)
    for line in a:
        x.append(float(line[0]))
        y.append(float(line[1]))        
# plot(x,y)

Populating the interactive namespace from numpy and matplotlib


In [39]:
# x, y - data to prosess, should be initialize befor

# I assume that  command below was executed, qt - to show plots in separate window, use 'inline' instead to plot here 
#%pylab qt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter
from inspect import getargspec

# Gaussian function
def gaussian(x,N,x0,sigma, background):
    """Returns value of a 1D-gaussian with the given parameters ,N,x0,sigma, background"""
    #from numpy import sqrt,pi,exp
    return N / (sigma * sqrt(pi)) * exp(-(x - x0)**2/(sigma**2)) + background

# lorentz function
def lorenz(x, a, x0, g, bg):
    return a * g/2 / ((x-x0)**2 + g**2/4) + bg

# If filtering is needed
y = gaussian_filter(y,20)

# threshold value to find piks and then to estimate width
thr = 0.04 #max(y)/2

# construct list of indexis y[index] > threshold
n1 = [x for (x,val) in enumerate(y) if val > thr] 
#print(n1)

# construct coordinates of beggining and ending of the pick based on threshold value
coords = []
for num,elem in enumerate(n1):
    if num == 0:   # iteration start
        st = 0
    elif elem - n1[num-1] > 400:    # if jump to next pick, number specifies minimum distance between peaks
        coords.append((n1[st],n1[num-1]))
        st = num
    elif  num == len(n1)-1:       # if came to list end
        coords.append((n1[st],n1[num]))
# print(coords)        

# fitting of individual peak in the neghboring of n * width and plotting them
n = 8     #what range to use to find fit
plot(x,y)
popts = []
# fit_func = lorenz
fit_func = gaussian
print('Fit parameters:',getargspec(fit_func)[0][1:])
for num,(lw,up) in enumerate(coords):
    width = abs(up - lw)
    sls = slice(max(0,lw - n*width), min(up + n*width,len(x)))
    popt, pcov = curve_fit(fit_func, x[sls], y[sls],p0=(0.001, (x[up]+x[lw])/2, (x[up]-x[lw]),0.023))
    plot(x[sls],fit_func(x[sls],*popt))
    popts.append(popt)
    annotate(str(num),xy=(popt[1],0))
    print(num, '---->', popt)
savefig('picks.png',dpi=300)

c:\program files\anaconda\lib\site-packages\ipykernel\__main__.py:47: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead


Fit parameters: ['N', 'x0', 'sigma', 'background']
0 ----> [ 0.0002658  -0.02078823  0.00045367  0.02431271]
1 ----> [ 0.00029076  0.00031567  0.00043247  0.02397259]
2 ----> [  2.84963408e-05   1.51187016e-02   5.12814808e-04   2.30149047e-02]
3 ----> [  3.28709848e-05   3.39998205e-02   4.87855168e-04   2.33392020e-02]
4 ----> [  3.41776602e-05   4.49527948e-02   4.82948261e-04   2.34394454e-02]
5 ----> [  3.12348832e-05   6.40213385e-02   5.44956129e-04   2.30136185e-02]
6 ----> [ 0.00028984  0.07856243  0.0003957   0.02399944]
7 ----> [ 0.00014024  0.09976447  0.00048284  0.02413931]
8 ----> [  2.80270865e-05   1.14605848e-01   5.16824150e-04   2.30521882e-02]
9 ----> [  2.33170183e-05   1.33504070e-01   5.96166918e-04   2.33432843e-02]
10 ----> [  2.44196893e-05   1.44479161e-01   5.33681094e-04   2.33418893e-02]
11 ----> [  3.19643377e-05   1.63525115e-01   5.40144772e-04   2.30755195e-02]


In [18]:
# to calculate relation between peak width and distance 
n0 = 1
n1 = 6
abs(popts[n0][1] - popts[n1][1]) / popts[n0][2] 

182.57118164869647

In [8]:
pwd

'D:\\!Data\\GIT\\ULE_410'